In [1]:
%%capture
from pathlib import Path

if Path.cwd().stem == "notebooks":
    %cd ..
    %load_ext autoreload
    %autoreload 2

In [ ]:
import logging
from pathlib import Path

import holoviews as hv
import hvplot.polars  # noqa
import polars as pl
from polars import col

from src.data.database_manager import DatabaseManager
from src.log_config import configure_logging
from src.plots.correlations import (
    aggregate_correlations_fisher_z,
    calculate_correlations_by_trial,
    plot_correlations_by_participant,
    plot_correlations_by_trial,
)

logger = logging.getLogger(__name__.rsplit(".", maxsplit=1)[-1])
configure_logging(
    stream_level=logging.DEBUG,
    ignore_libs=["matplotlib", "Comm", "bokeh", "tornado", "param", "numba"],
)

pl.Config.set_tbl_rows(12)  # for the 12 trials
hv.output(widget_location="bottom", size=130)

In [ ]:
db = DatabaseManager()
with db:
    df = db.get_trials("Explore_Data", exclude_problematic=True)

# Rename columns
df = df.rename(
    {
        "rating": "pain_rating",
    }
)

# Remove first 20 seconds
df = df.filter(col("normalized_timestamp") >= 20 * 1000)
df

trial_id,trial_number,participant_id,timestamp,temperature,pain_rating,eda_raw,eda_tonic,eda_phasic,eda_tonic_detrended,ppg_raw,heart_rate,ibi,pupil_l_raw,pupil_r_raw,pupil_r,pupil_l,pupil,brow_furrow,cheek_raise,mouth_open,upper_lip_raise,nose_wrinkle,normalized_timestamp,stimulus_seed,skin_patch,decreasing_intervals,major_decreasing_intervals,increasing_intervals,strictly_increasing_intervals,strictly_increasing_intervals_without_plateaus,plateau_intervals,prolonged_minima_intervals
u16,u8,u8,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,u16,u8,u16,u16,u16,u16,u16,u16,u16
1,1,1,220180.9348,0.934688,1.0,19.102174,19.371588,-0.269414,1.243299,1350.741863,79.898579,-3.189252,4.255447,4.021717,4.030407,4.271057,4.150732,0.318907,0.000037,0.000071,0.000167,0.00117,20000.0,870,1,1,1,0,0,0,0,0
1,1,1,220280.9348,0.932326,1.0,19.137027,19.364372,-0.227345,1.235748,1342.388671,79.61188,2.952829,4.246915,4.007104,4.030419,4.270214,4.150317,0.313007,0.000037,0.00007,0.000165,0.001149,20100.0,870,1,1,1,0,0,0,0,0
1,1,1,220380.9348,0.929957,1.0,19.192711,19.354214,-0.161504,1.225124,1403.603244,78.897245,-4.026874,4.245434,4.002811,4.030761,4.271087,4.150924,0.304226,0.000037,0.000068,0.000162,0.001118,20200.0,870,1,1,1,0,0,0,0,0
1,1,1,220480.9348,0.927382,1.0,19.256503,19.345154,-0.088651,1.215651,1464.013352,78.005557,17.333124,4.253932,4.010675,4.029863,4.272534,4.151198,0.294819,0.000037,0.000066,0.000158,0.001086,20300.0,870,1,1,1,0,0,0,0,0
1,1,1,220580.9348,0.924825,1.0,19.317642,19.335727,-0.018085,1.205797,1431.159865,76.923417,50.017231,4.268048,4.035022,4.039995,4.272243,4.156119,0.285137,0.000037,0.000063,0.000156,0.001054,20400.0,870,1,1,1,0,0,0,0,0
1,1,1,220680.9348,0.922063,1.0,19.371235,19.327207,0.044028,1.196895,1406.549896,75.977334,11.420777,4.295428,4.060578,4.056849,4.296271,4.17656,0.275081,0.000037,0.000061,0.000153,0.001022,20500.0,870,1,1,1,0,0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
496,4,42,1.1405e6,0.031406,0.0,15.030559,15.008543,0.022016,-2.603602,1434.106413,62.353678,-1.19803,3.577106,3.476652,3.50111,3.600631,3.550871,0.015639,0.000177,0.139924,0.002577,0.002907,179500.0,806,3,2395,1437,0,0,0,0,0
496,4,42,1.1406e6,0.030945,0.0,15.025569,15.006031,0.019538,-2.606329,1412.519621,62.199718,-0.764415,3.548026,3.485169,3.49922,3.594389,3.546804,0.016379,0.000181,0.145722,0.002668,0.002891,179600.0,806,3,2395,1437,0,0,0,0,0


In [6]:
facial_features = [
    "brow_furrow",
    "brow_furrow",
    "cheek_raise",
    "mouth_open",
    "upper_lip_raise",
    "nose_wrinkle",
]


In [7]:
df.filter(participant_id=35, trial_number=7)

trial_id,trial_number,participant_id,timestamp,temperature,pain_rating,eda_raw,eda_tonic,eda_phasic,eda_tonic_detrended,ppg_raw,heart_rate,ibi,pupil_l_raw,pupil_r_raw,pupil_r,pupil_l,pupil,brow_furrow,cheek_raise,mouth_open,upper_lip_raise,nose_wrinkle,normalized_timestamp,stimulus_seed,skin_patch,decreasing_intervals,major_decreasing_intervals,increasing_intervals,strictly_increasing_intervals,strictly_increasing_intervals_without_plateaus,plateau_intervals,prolonged_minima_intervals
u16,u8,u8,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,u16,u8,u16,u16,u16,u16,u16,u16,u16
415,7,35,1.6100e6,0.463265,0.239638,28.593798,28.591943,0.001856,0.621688,1368.402459,57.817304,0.608313,3.511246,3.993089,3.994384,3.510105,3.752244,0.000029,0.000044,0.020821,2.6687e-10,0.000011,20000.0,396,1,0,0,1996,0,0,799,0
415,7,35,1.6101e6,0.463265,0.24,28.589304,28.587536,0.001768,0.616997,1331.960626,58.001909,-1.858516,3.509041,3.991996,3.994931,3.511168,3.75305,0.000029,0.000044,0.020775,3.0122e-10,0.000011,20100.0,396,1,0,0,1996,0,0,799,0
415,7,35,1.6102e6,0.463265,0.24,28.579791,28.583585,-0.003794,0.612789,1298.805648,58.152984,-1.529351,3.506133,3.997522,4.000028,3.511844,3.755936,0.000029,0.000043,0.020706,3.6420e-10,0.000011,20200.0,396,1,0,0,1996,0,0,799,0
415,7,35,1.6103e6,0.463265,0.24,28.576887,28.58054,-0.003654,0.609544,1263.109421,58.250291,2.340215,3.519214,4.003781,4.004771,3.520813,3.762792,0.000029,0.000043,0.020628,4.5313e-10,0.000011,20300.0,396,1,0,0,1996,0,0,799,0
415,7,35,1.6104e6,0.463265,0.24,28.594749,28.57525,0.019499,0.603902,1531.71922,58.354854,-4.511113,3.555263,4.024007,4.023473,3.55591,3.789692,0.000029,0.000042,0.020564,5.4656e-10,0.000011,20400.0,396,1,0,0,1996,0,0,799,0
415,7,35,1.6105e6,0.463265,0.24,28.589431,28.57107,0.018361,0.599441,1596.010568,58.377827,2.882281,3.598577,4.051377,4.041449,3.594781,3.818115,0.000028,0.000041,0.020486,6.9287e-10,0.000011,20500.0,396,1,0,0,1996,0,0,799,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
415,7,35,1.7695e6,0.277676,0.453085,31.428516,31.255031,0.173485,2.837621,1240.710182,69.210855,-1.119446,3.288059,3.672028,3.673095,3.287548,3.480321,0.000002,0.000467,0.033616,3.3879e-7,0.000014,179500.0,396,1,2000,0,0,0,0,0,0
415,7,35,1.7696e6,0.276715,0.399274,31.404094,31.244098,0.159996,2.826408,1223.750477,69.228758,-0.969774,3.30334,3.715308,3.716203,3.302653,3.509428,0.000002,0.000455,0.034276,4.1173e-7,0.000014,179600.0,396,1,2000,0,0,0,0,0,0


# Temperature

In [8]:
COLORS = {
    "temperature_brow_furrow_corr": "red",
    "temperature_cheek_raise_corr": "#2ca02c",
    "temperature_mouth_open_corr": "#d62728",
    "temperature_upper_lip_raise_corr": "#9467bd",
    "temperature_nose_wrinkle_corr": "#ff7f0e",
}

In [9]:
corrs_by_participant = {}
plots = {}

for feature in facial_features:
    col1, col2 = "temperature", feature

    corr_by_trial = calculate_correlations_by_trial(df, col1, col2)
    corrs_by_participant[feature] = aggregate_correlations_fisher_z(
        corr_by_trial, col1, col2, "participant_id", include_ci=True
    )
    plot_correlations_by_trial(corr_by_trial, col1, col2)
    plots[feature] = plot_correlations_by_participant(
        corrs_by_participant[feature], col1, col2, with_config=False, color_map=COLORS
    )

combined_chart = (
    plots["brow_furrow"]
    + plots["cheek_raise"]
    + plots["mouth_open"]
    + plots["upper_lip_raise"]
    + plots["nose_wrinkle"]
)
combined_chart

alt.LayerChart(...)

In [10]:
corrs_by_participant["brow_furrow"].describe()

statistic,participant_id,participant_id_temperature_brow_furrow_corr_mean,participant_id_temperature_brow_furrow_corr_ci_lower,participant_id_temperature_brow_furrow_corr_ci_upper
str,f64,f64,f64,f64
"""count""",42.0,42.0,42.0,42.0
"""null_count""",0.0,0.0,0.0,0.0
"""mean""",21.5,0.060127,-0.05822,0.176579
"""std""",12.267844,0.163192,0.174634,0.165699
"""min""",1.0,-0.22376,-0.38765,-0.138456
"""25%""",11.0,-0.080416,-0.177422,0.047064
"""50%""",22.0,0.072795,-0.05892,0.179473
"""75%""",32.0,0.141205,0.047289,0.296635
"""max""",42.0,0.450871,0.355712,0.536777


In [11]:
# Save figure
import os
from pathlib import Path

from dotenv import load_dotenv

load_dotenv()
FIGURE_DIR = Path(os.getenv("FIGURE_DIR"))


# Save the figure
path = FIGURE_DIR / "correlations_with_temperature_face.png"
combined_chart.save(path)
# Or save as SVG for vector graphics:
# combined_chart.save(FIGURE_DIR / "correlations_with_temperature.svg")

# Rating

In [12]:
COLORS = {
    "pain_rating_brow_furrow_corr": "red",
    "pain_rating_cheek_raise_corr": "#2ca02c",
    "pain_rating_mouth_open_corr": "#d62728",
    "pain_rating_upper_lip_raise_corr": "#9467bd",
    "pain_rating_nose_wrinkle_corr": "#ff7f0e",
}

In [13]:
plots = {}

for feature in facial_features:
    col1, col2 = "pain_rating", feature

    corr_by_trial = calculate_correlations_by_trial(df, col1, col2)
    corr_by_participant = aggregate_correlations_fisher_z(
        corr_by_trial, col1, col2, "participant_id", include_ci=True
    )
    plot_correlations_by_trial(corr_by_trial, col1, col2)
    plots[feature] = plot_correlations_by_participant(
        corr_by_participant, col1, col2, with_config=False, color_map=COLORS
    )

combined_chart = (
    plots["brow_furrow"]
    + plots["cheek_raise"]
    + plots["mouth_open"]
    + plots["upper_lip_raise"]
    + plots["nose_wrinkle"]
)
combined_chart

alt.LayerChart(...)

In [14]:
# Save figure
import os
from pathlib import Path

from dotenv import load_dotenv

load_dotenv()
FIGURE_DIR = Path(os.getenv("FIGURE_DIR"))


# Save the figure
path = FIGURE_DIR / "correlations_with_rating_face.png"
combined_chart.save(path)
# Or save as SVG for vector graphics:
# combined_chart.save(FIGURE_DIR / "correlations_with_temperature.svg")